# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションに参加するための機械学習モデルの構築と推論プロセスに焦点を当てています。以下にノートブックの要約を示します。

### 問題の概要
本ノートブックの目的は、異なる大規模言語モデル（LLM）からの応答を比較して、どちらのモデルの応答がユーザーに好まれるかを予測することです。このタスクは、人間の好みを予測するためのモデルの開発を通じて、LLMとの相互作用を改善しようとしています。

### 手法
1. **ライブラリの使用**:
   - **PyTorch**: ニューラルネットワークの構築とトレーニングに使用。
   - **Transformers**: 事前訓練済みのモデルのロードとトークナイザーの利用に使用。
   - **Datasets**: データのロードや前処理に使用。
   - **Accelerate**: モデルを効率的にトレーニングするために使用し、マルチGPUトレーニングに対応しています。

2. **モデルアーキテクチャ**:
   - **Phi-3 miniモデル**: Microsoftから提供される事前訓練済みのモデルを使用し、LoRA（Low-Rank Adaptation）を利用してモデルを微調整しています。
   - カスタムデータセットクラスとデータローダーを定義し、トークン化されたプロンプトと応答を準備します。

3. **トレーニング手順**:
   - ハイパーパラメータ（最大シーケンス長やバッチサイズなど）を設定。
   - LoRAの適用や量子化を通じてモデルの効率を向上。
   - 自動混合精度を用いて、推論中に計算リソースを最適化。

4. **データの前処理**:
   - テストデータセットを読み込み、プロンプトと応答をJSONから抽出。
   - データを2つのバッチに分割し、各GPUで別々に推論を実行。

5. **推論と結果の取得**:
   - モデルを評価モードに設定し、推論を行う関数を使用して結果を収集。
   - 複数のスレッドを用いて推論を並列実行し、処理時間を短縮。

6. **結果の保存**:
   - 予測結果を含むデータフレームを生成し、最終的な提出フォーマットに変換してCSVファイルとして保存。
   - 不要なデータやファイルの削除を行い、リソースを効率的に管理。

### 結論
このノートブックは、LMSYS Chatbot Arenaコンペティションにおける人間の好み予測モデルの構築を目的としており、効率的なトレーニングと推論の手法を採用しています。PyTorchとTransformersライブラリを用いて、LoRAトレーニングや量子化を組み合わせることで、モデルの性能を向上させています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する専門用語の簡単な解説を示します。これらは初心者にとって馴染みが薄い可能性があるため、注意が必要です。

1. **LoRA (Low-Rank Adaptation)**:
   LoRAは、大規模モデルのパラメータを凍結したまま、少ないパラメータで適応させる手法です。モデルの重みを低ランク行列で補正することで、効率的に学習ができるようになります。

2. **量子化**:
   モデルのサイズを小さくし、推論速度を向上させるために、フロート数値を整数に変換するプロセスです。量子化により、メモリ使用量が減少し、デバイス上での計算が効率化されます。

3. **アテンションマスク**:
   シーケンスデータを処理する際に、特定の部分に注意を集中させるために用いるマスクです。無関係なトークンに対して注意を払わないようにするために使用され、通常、0または1の値で表されます。

4. **Rotary Position Embedding (RoPE)**:
   位置情報をモデルに追加する手法の一つで、特にTransformerアーキテクチャで使用されます。RoPEは、トークンの相対的位置関係を学習するために、回転行列を利用して位置情報を表現します。

5. **因果関係 (Causal Relation)**:
   モデルが過去の情報に基づいて未来を予測することを指します。具体的には、系列データを処理する際、未来の情報にアクセスできないようにすることで、信頼性のある予測を行います。

6. **ソフトマックス**:
   多クラスの出力を確率分布に変換するための関数です。各クラスの出力を指数関数に通して、全体の合計が1になるように正規化し、確率として解釈できるようにします。

7. **データコレータ (Data Collator)**:
   モデルのトレーニング中に、バッチ単位でデータを整理・整形するための関数やクラスです。特に異なる長さのシーケンスを適切にパディングしてバッチを構成することに役立ちます。

8. **バッチサイズ**:
   モデルに入力されるデータのグループ数を指します。トレーニングや推論時にデータを一度に処理するための最小単位となります。

9. **パディング**:
   シーケンスデータの長さを揃えるために、短いシーケンスに特定の値（通常は0）を追加するプロセスです。これにより、計算を効率よく行えるようになります。

10. **クロスエントロピー損失 (CrossEntropyLoss)**:
    多クラス分類問題で用いられる損失関数です。モデルの予測確率と実際のラベルの違いを測定し、訓練中に最小化することを目指します。

これらの用語は、ノートブック内で使用されるコンテキスト特有のものであり、初心者の理解を助けるために特に重要です。

---


# インストール不要
microsoft/Phi-3-mini-4k-instruct + LoRA > GPU並列トレーニング

最大シーケンス長はモデルのパフォーマンスに大きな影響を与えますが、メモリ不足のため最大長は768に設定されました。

[トレーニングコード](https://www.kaggle.com/code/argozero01/parallel-train-phi-3-mini-4k-instruct)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import datasets
from datasets import load_dataset, load_metric, Dataset

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, log_loss

from accelerate import notebook_launcher, Accelerator, PartialState
from accelerate.utils import write_basic_config
from accelerate.inference import prepare_pippy

import transformers
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    set_seed,
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    AutoConfig
)

import os
import shutil
import math
import json
from tqdm import tqdm
import gc
import pandas as pd
import numpy as np
from typing import Optional, Tuple

# ここでは、PyTorchと関連ライブラリをインポートしています。
# Torchを使ってニューラルネットワークモデルを構築し、トレーニングや評価を行う準備をします。

In [ ]:
# ハイパーパラメータの設定
model_name = "/kaggle/input/microsoftphi-3-mini-4k-instruct/transformers/default/1"  # モデルの名前
model_path = "/kaggle/input/checkpoint-phi3/model_checkpoint.pth"  # モデルのチェックポイントのパス
seed = 42  # 乱数シード
lora_r = 2  # LoRAのランク
quantize_bit = 16  # 量子化ビット数
test_batch_size = 1  # テスト時のバッチサイズ
test_max_len = 256  # テスト時の最大シーケンス長
device = "cuda"  # 使用するデバイス（GPU）

# これらの変数はモデルの設定やトレーニング設定のために使用されます。

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    # カスタムデータセットクラスの定義

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer  # トークナイザーの初期化
        self.prompt = df['prompt']  # データフレームからプロンプトを取得
        self.response_a = df['response_a']  # データフレームから応答Aを取得
        self.response_b = df['response_b']  # データフレームから応答Bを取得
        self.max_len = max_len  # 最大シーケンス長
        self.targets = df.get('labels', None)  # ラベルの取得（存在すれば）

    def __len__(self):
        return len(self.prompt)  # データセットのサンプル数を返す

    def __getitem__(self, index):
        # 指定したインデックスのデータを取得
        
        prompt = str(self.prompt[index])  # プロンプトを文字列として取得
        response_a = str(self.response_a[index])  # 応答Aを文字列として取得
        response_b = str(self.response_b[index])  # 応答Bを文字列として取得

        prompt_len = len(self.tokenizer("##prompt: " + prompt, add_special_tokens=True)['input_ids'])  # プロンプトのトークン数を計算
        response_a_len = len(self.tokenizer("##response_a: " + response_a, add_special_tokens=True)['input_ids'])  # 応答Aのトークン数を計算
        response_b_len = len(self.tokenizer("##response_b: " + response_b, add_special_tokens=True)['input_ids'])  # 応答Bのトークン数を計算

        final_prompt_len = min(self.max_len, prompt_len)  # プロンプトの最終長を決定
        final_a_len = min(self.max_len, response_a_len)  # 応答Aの最終長を決定
        final_b_len = min(self.max_len, response_b_len)  # 応答Bの最終長を決定

        # トークン化とパディングを行い、入力IDとアテンションマスクを作成
        prompt_token = self.tokenizer("##prompt: " + prompt, add_special_tokens=True, max_length=final_prompt_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_a_token = self.tokenizer("##response_a: " + response_a, add_special_tokens=True, max_length=final_a_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_b_token = self.tokenizer("##response_b: " + response_b, add_special_tokens=True, max_length=final_b_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')

        # 入力IDとアテンションマスクを結合
        input_ids = torch.cat([prompt_token['input_ids'], response_a_token['input_ids'], response_b_token['input_ids']], dim=1)
        attention_mask = torch.cat([prompt_token['attention_mask'], response_a_token['attention_mask'], response_b_token['attention_mask']], dim=1)

        if self.targets is not None:
            labels = torch.LongTensor([self.targets[index]])  # ラベルをTensorに変換
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten(), 'labels': labels}  # データを辞書形式で返す
        else:
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten()}  # ラベルがない場合はラベルなしでデータを返す

In [ ]:
def custom_collate_fn(batch, tokenizer):
    # カスタムコレート関数の定義

    input_ids = [item['input_ids'] for item in batch]  # バッチ内の入力IDを取得
    attention_masks = [item['attention_mask'] for item in batch]  # バッチ内のアテンションマスクを取得
    labels = torch.cat([item['labels'] for item in batch], dim=0) if 'labels' in batch[0] else None  # ラベルが含まれている場合は結合

    # バッチ内のシーケンスの最大長を見つける
    max_len = max([input_id.size(0) for input_id in input_ids])

    # 新しい最大長で再トークン化
    new_input_ids = []
    new_attention_masks = []

    for item in batch:
        # 各アイテムのデータを新しい最大長に合わせて切り詰め
        input_ids = item['input_ids'][:max_len]
        attention_mask = item['attention_mask'][:max_len]

        new_input_ids.append(input_ids)  # 新しい入力IDを追加
        new_attention_masks.append(attention_mask)  # 新しいアテンションマスクを追加

    # パディングを施してテンソルを作成
    new_input_ids = pad_sequence(new_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    new_attention_masks = pad_sequence(new_attention_masks, batch_first=True, padding_value=0)

    output = {
        'input_ids': new_input_ids,
        'attention_mask': new_attention_masks
    }

    if labels is not None:
        output['labels'] = labels  # ラベルがある場合は出力に追加

    return output  # 結果を返す

In [ ]:
def create_dataloaders(df, tokenizer, max_len, batch_size, shuffle=True):
    # データローダーを作成する関数の定義
    dataloader = DataLoader(
        CustomDataset(df, tokenizer, max_len),  # カスタムデータセットを使ったデータローダー
        shuffle=shuffle,  # シャッフルオプション
        batch_size=batch_size,  # バッチサイズ
        collate_fn=lambda x: custom_collate_fn(x, tokenizer)  # コレート関数を指定
    )
    return dataloader  # データローダーを返す

In [ ]:
def quantize_tensor(tensor, num_bits=quantize_bit):
    # テンソルを量子化する関数の定義
    qmin = 0.
    qmax = 2.**num_bits - 1.

    min_val, max_val = tensor.min(), tensor.max()  # テンソルの最小値と最大値を取得
    scale = (max_val - min_val) / (qmax - qmin)  # スケールを計算
    zero_point = qmin - min_val / scale  # ゼロポイントを計算

    quantized_tensor = torch.round(tensor / scale + zero_point)  # テンソルを量子化
    quantized_tensor = torch.clamp(quantized_tensor, qmin, qmax)  # クランプ処理
    quantized_tensor = (quantized_tensor - zero_point) * scale  # スケールを戻して再スケーリング

    return quantized_tensor  # 量子化されたテンソルを返す

def quantize_model(model, num_bits=8):
    # モデル全体を量子化する関数の定義
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            module.weight.data = quantize_tensor(module.weight.data, num_bits)  # 重みを量子化
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)  # バイアスを量子化
        elif isinstance(module, nn.Conv2d):
            module.weight.data = quantize_tensor(module.weight.data, num_bits)  # 畳み込み層の重みを量子化
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)  # 畳み込み層のバイアスを量子化

    return model  # 量子化されたモデルを返す

# import torch.quantization

# def quantize_model_dynamic(model):
#     model.qconfig = torch.quantization.default_dynamic_qconfig
#     torch.quantization.prepare(model, inplace=True)
#     torch.quantization.convert(model, inplace=True)
#     return model

In [ ]:
class LoRA(nn.Module):
    # LoRAクラスの定義
    def __init__(self, in_features, out_features, rank=lora_r, alpha=1.0, lora_dropout=0.05):
        super(LoRA, self).__init__()
        self.alpha = alpha  # スケーリングファクタ
        self.rank = rank  # LoRAのランク
        self.lora_a = nn.Linear(in_features, rank, bias=False)  # 変換A
        self.lora_b = nn.Linear(rank, out_features, bias=False)  # 変換B
        self.dropout = nn.Dropout(lora_dropout)  # ドロップアウト層

    def forward(self, x):
        lora_out = self.alpha * self.lora_b(self.lora_a(x))  # LoRAの出力を計算
        lora_out = self.dropout(lora_out)  # ドロップアウトを適用
        return lora_out  # 出力を返す

In [ ]:
from transformers.models.phi3.modeling_phi3 import (
    Phi3RotaryEmbedding,
    # Phi3LongRoPEScaledRotaryEmbedding,  # コメントアウトされたコード
    apply_rotary_pos_emb,
    repeat_kv
)

# Phi3RotaryEmbeddingなどのモジュールをインポートしています。
# これにより、ロタリー埋め込みや位置埋め込みの適用を行えるようになります。

In [ ]:
class Phi3Attention(nn.Module):
    """マルチヘッドアテンション 'Attention Is All You Need' 論文から"""

    def __init__(self, config, layer_idx: Optional[int] = None):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        if layer_idx is None:
            logger.warning_once(
                f"layer_idxを渡さずに{self.__class__.__name__}をインスタンス化することは推奨されず、"
                "キャッシングを使用する際にフォワードコール中にエラーが発生します。必ず`layer_idx`を指定してこのクラスを作成してください。"
            )

        self.attention_dropout = config.attention_dropout  # アテンションドロップアウト率
        self.hidden_size = config.hidden_size  # 隠れ層のサイズ
        self.num_heads = config.num_attention_heads  # アテンションヘッドの数
        self.head_dim = self.hidden_size // self.num_heads  # 各ヘッドの次元数
        self.num_key_value_heads = config.num_key_value_heads  # キー・バリューのヘッド数
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads  # キー・バリューのグループ数
        self.max_position_embeddings = config.max_position_embeddings  # 最大位置埋め込み
        self.original_max_position_embeddings = config.original_max_position_embeddings
        self.rope_theta = config.rope_theta  # RoPEのスケーリングパラメータ
        self.rope_scaling = config.rope_scaling  # RoPEのスケーリング方法
        self.is_causal = True  # 因果関係を持つかどうか

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_sizeはnum_headsで割り切れる必要があります（`hidden_size`: {self.hidden_size}"
                f" と `num_heads`: {self.num_heads}を受け取りました）。"
            )

        op_size = self.num_heads * self.head_dim + 2 * (self.num_key_value_heads * self.head_dim)  # 出力サイズ
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)  # 出力の線形変換
        self.qkv_proj = nn.Linear(self.hidden_size, op_size, bias=False)  # QKVの線形変換
        self._init_rope()  # RoPEの初期化

        ########################## LoRAアダプタ ##########################
        self.qkv_lora = LoRA(self.hidden_size, op_size, lora_r)  # QKVのLoRAアダプタ
        self.o_lora = LoRA(self.num_heads * self.head_dim, self.hidden_size, lora_r)  # 出力のLoRAアダプタ
        ########################## LoRAアダプタ ##########################

    def _init_rope(self):
        # RoPEの初期化
        if self.rope_scaling is None:
            self.rotary_emb = Phi3RotaryEmbedding(
                self.head_dim,
                max_position_embeddings=self.max_position_embeddings,
                base=self.rope_theta,
            )
        else:
            scaling_type = self.config.rope_scaling["type"]
            if scaling_type == "longrope":
                self.rotary_emb = Phi3LongRoPEScaledRotaryEmbedding(self.head_dim, self.config)
            else:
                raise ValueError(f"未知のRoPEスケーリングタイプ {scaling_type}")

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        # logger.warning_once("フラッシュアテンション実装を実行していないため、数値の違いが予想されます。")

        bsz, q_len, _ = hidden_states.size()  # バッチサイズとクエリ長を取得
        ########################## LoRAアダプタ ##########################
        qkv = self.qkv_proj(hidden_states) + self.qkv_lora(hidden_states)  # QKVプロジェクションの計算
        ########################## LoRAアダプタ ##########################
        
        query_pos = self.num_heads * self.head_dim  # クエリ位置の計算
        query_states = qkv[..., :query_pos]  # クエリ状態の取得
        key_states = qkv[..., query_pos : query_pos + self.num_key_value_heads * self.head_dim]  # キー状態の取得
        value_states = qkv[..., query_pos + self.num_key_value_heads * self.head_dim :]  # バリュー状態の取得

        # 状態を再構成
        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = key_states.shape[-2]  # キー・バリューのシーケンス長
        if past_key_value is not None:
            if self.layer_idx is None:
                raise ValueError(
                    f"キャッシュ構造はバージョンv4.36以降変更されました。k/vキャッシングを使用するために{self.__class__.__name__}を使用している場合、"
                    "レイヤーインデックスでこのアテンションクラスを初期化してください。"
                )
            kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)  # 過去のキー・バリューの長さを取得
        cos, sin = self.rotary_emb(value_states, position_ids, seq_len=kv_seq_len)  # RoPEの計算

        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)  # RoPEの適用

        if past_key_value is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # RoPEモデルに特有のキーワード引数
            key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)  # キャッシュの更新

        # n_kv_heads < n_headsの場合にk/vヘッドを繰り返す
        key_states = repeat_kv(key_states, self.num_key_value_groups)  # キー状態の繰り返し
        value_states = repeat_kv(value_states, self.num_key_value_groups)  # バリュー状態の繰り返し

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)  # アテンション重みの計算

        if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
            raise ValueError(
                f"アテンション重みは{(bsz, self.num_heads, q_len, kv_seq_len)}のサイズであるべきですが、"
                f"{attn_weights.size()}になっています。"
            )

        if attention_mask is not None:
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]  # 因果マスクの取得
            attn_weights += causal_mask  # マスクの適用

        # アテンションをfp32にアップキャスト
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(value_states.dtype)  # ソフトマックスの適用
        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)  # ドロップアウトの適用

        attn_output = torch.matmul(attn_weights, value_states)  # アテンションの出力計算

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"attn_outputは{(bsz, self.num_heads, q_len, self.head_dim)}のサイズであるべきですが、"
                f"{attn_output.size()}になっています。"
            )

        attn_output = attn_output.transpose(1, 2).contiguous()  # 転置と連続性の保証
        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)  # 隠れ層のサイズに変形
        ########################## LoRAアダプタ ##########################
        attn_output = self.o_proj(attn_output) + self.o_lora(attn_output)  # 出力プロジェクションとLoRAアダプタの適用
        ########################## LoRAアダプタ ##########################
        
        if not output_attentions:
            attn_weights = None  # アテンション重みを返さない場合はNoneを設定

        return attn_output, attn_weights, past_key_value  # 結果を返す

In [ ]:
def replace_attention_module(config, layer, layer_idx):
    # アテンションモジュールを置き換える関数
    if hasattr(layer, 'self_attn') and layer_idx > 12:  # レイヤーがself_attnを持ち、レイヤーインデックスが12より大きい場合

        new_attention = Phi3Attention(config, layer_idx)  # 新しいアテンションを作成

        # ウェイトのコピー
        new_attention.qkv_proj.weight.data.copy_(layer.self_attn.qkv_proj.weight.data)
        new_attention.o_proj.weight.data.copy_(layer.self_attn.o_proj.weight.data)

        layer.self_attn = new_attention  # 置き換え

In [ ]:
loss_fn = nn.CrossEntropyLoss()  # クロスエントロピー損失関数の定義

class LoraModelForClassification(nn.Module):
    def __init__(self, lora_model):  # LoRAモデルの初期化
        super(LoraModelForClassification, self).__init__()
        self.config = lora_model.config  # モデルの設定
        self.peft_model = lora_model  # LoRAモデルの保存
        self.dropout = nn.Dropout(0.1)  # ドロップアウト層
        self.classifier = nn.Linear(self.config.hidden_size, 3)  # 分類器の定義（3クラス分類）

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.peft_model(input_ids, attention_mask=attention_mask)  # LoRAモデルのフォワードパス
        pooled_output = outputs.last_hidden_state.mean(dim=1)  # プーリング操作
        output_dropout = self.dropout(pooled_output)  # ドロップアウト適用
        logits = self.classifier(output_dropout)  # 分類器による出力

        loss = None
        if labels is not None:
            loss = loss_fn(logits, labels)  # ラベルが存在する場合は損失を計算
        return loss, logits  # 損失とロジットを返す

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータの読み込み
len(test)  # テストデータのサンプル数を表示

In [ ]:
import json
# JSON形式の文字列をデータフレームの各カラムに適用する
test["prompt"] = test["prompt"].apply(lambda x: json.loads(x)[0])  # プロンプトの抽出
test["response_a"] = test["response_a"].apply(lambda x: json.loads(x)[0])  # 応答Aの抽出
test["response_b"] = test["response_b"].apply(lambda x: json.loads(x)[0])  # 応答Bの抽出

In [ ]:
test_0 = test[:len(test)//2].reset_index(drop=True)  # テストデータの前半を取得
test_1 = test[len(test)//2:].reset_index(drop=True)  # テストデータの後半を取得

In [ ]:
from torch.cuda.amp import autocast

def infer(model, dataloader, device):
    # モデルを評価モードに設定
    model.eval()

    target_list = []

    for batch in dataloader:
        with torch.no_grad():
            with autocast():  # 自動混合精度の適用
                input_ids = batch["input_ids"].to(device)  # 入力IDをデバイスに転送
                attention_mask = batch["attention_mask"].to(device)  # アテンションマスクをデバイスに転送
                _, logits = model(input_ids=input_ids, attention_mask=attention_mask)  # モデル推論
                softmax_logits = torch.nn.functional.softmax(logits, dim=1)  # ソフトマックスを適用
                target_list.append(softmax_logits)  # 結果をリストに追加

    return target_list  # 推論結果を返す

In [ ]:
from threading import Thread

gpu0 = "cuda:0"  # 使用するGPUデバイス
gpu1 = "cuda:1"  # 使用するGPUデバイス

In [ ]:
model0 = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16
                                  , device_map="cpu")  # モデルのロード
model0 = quantize_model(model0)  # モデルの量子化
for idx, layer in enumerate(model0.layers):
    replace_attention_module(model0.config, layer, idx)  # アテンションモジュールの置き換え
model0 = LoraModelForClassification(model0)  # LoRAモデルの構築

model1 = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16
                                  , device_map="cpu")  # モデルのロード
model1 = quantize_model(model1)  # モデルの量子化
for idx, layer in enumerate(model1.layers):
    replace_attention_module(model1.config, layer, idx)  # アテンションモジュールの置き換え
model1 = LoraModelForClassification(model1)  # LoRAモデルの構築


model0.load_state_dict(torch.load(model_path))  # モデルの重みをロード
model1.load_state_dict(torch.load(model_path))  # モデルの重みをロード
model0.to(gpu0)  # GPUデバイスにモデルを転送
model1.to(gpu1)  # GPUデバイスにモデルを転送

In [ ]:
tokenizer0 = AutoTokenizer.from_pretrained(model_name)  # トークナイザーのロード

if tokenizer0.pad_token is None:
    tokenizer0.pad_token = tokenizer0.eos_token  # パディングトークンの設定
tokenizer0.padding_side = "right"  # fp16トレーニングでの変 overflow問題を修正

tokenizer1 = AutoTokenizer.from_pretrained(model_name)  # トークナイザーのロード

if tokenizer1.pad_token is None:
    tokenizer1.pad_token = tokenizer1.eos_token  # パディングトークンの設定
tokenizer1.padding_side = "right"  # fp16トレーニングでの変 overflow問題を修正

test_dataloader0 = create_dataloaders(test_0, tokenizer0, test_max_len, test_batch_size, shuffle=False)  # データローダの作成
test_dataloader1 = create_dataloaders(test_1, tokenizer1, test_max_len, test_batch_size, shuffle=False)  # データローダの作成

In [ ]:
def run_inference(model, dataloader, device, results, index):
    # 推論処理を別スレッドで実行
    results[index] = infer(model, dataloader, device)  # 推論結果を格納

results = {}

process0 = Thread(target=run_inference, args=(model0, test_dataloader0, gpu0, results, 0))  # スレッドの作成
process1 = Thread(target=run_inference, args=(model1, test_dataloader1, gpu1, results, 1))  # スレッドの作成

# プロセスを開始
process0.start()
process1.start()

# 両方のプロセスが終了するのを待つ
process0.join()
process1.join()

In [ ]:
device = 'cuda:0'  # 移動するデバイスを指定
for k, v in results.items():
    for i in range(len(v)):
        results[k][i] = v[i].to(device)  # 結果を指定デバイスに転送

# 辞書の値を一つにまとめる
target_list = torch.cat([torch.cat(v, dim=0) for v in results.values()], dim=0)  # 結果を結合

In [ ]:
sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出ファイルの読み込み

In [ ]:
df_list = []  # データフレームを格納するリストを初期化
for tensor in target_list:
    # テンソルをデータフレームに変換
    df = pd.DataFrame(tensor.unsqueeze(0).detach().cpu().numpy(), columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
    df_list.append(df)  # データフレームをリストに追加

combined_df = pd.concat(df_list, axis=0, ignore_index=True)  # リスト内のデータフレームを結合

sub = sub.set_index(pd.Index(combined_df.index))  # インデックスを設定

final_df = pd.concat([sub[['id']], combined_df], axis=1)  # ID列と結果を結合

In [ ]:
def delete_files_and_folders(path):
    # 指定したパスが存在するか確認
    if not os.path.exists(path):
        print(f"エラー: {path} は存在しません。")
        return

    # パス内のすべてのファイルとフォルダを走査
    for root, dirs, files in os.walk(path, topdown=False):
        # ファイルを削除
        for name in files:
            if name == "submission.csv":
                print(f"スキップするファイル: {os.path.join(root, name)}")
                continue
            file_path = os.path.join(root, name)  # ファイルパスを組み立て
            print(f"削除するファイル: {file_path}")  # 削除するファイルを出力
            os.remove(file_path)  # ファイルを削除

    print(f"{path}内のすべてのファイルとフォルダが削除されました。")  # 削除完了メッセージ

# 例としてのパス
path_to_delete = "/kaggle/working/"  # 削除対象のパス

# ファイルとフォルダ削除関数を呼び出し
delete_files_and_folders(path_to_delete)

In [ ]:
final_df.to_csv('submission.csv', index=False)  # 結果をCSVファイルとして保存

In [ ]:
final_df.head()  # データフレームの先頭を表示

In [ ]:
# GPUメモリをクリアする関数
def clear_gpu_memory():
    torch.cuda.empty_cache()  # GPUメモリのキャッシュをクリア
    gc.collect()  # ガベージコレクタを呼び出し

# 学習後にGPUメモリをクリア
clear_gpu_memory()  # メモリをクリアする関数を実行